In [ ]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'ClusterCast.settings'
import test_utils
import django
django.setup()
import sys
sys.path.append("/home/ajp031/StockDeepLearning/ClusterCast/ClusterCast")
from asgiref.sync import sync_to_async
from importlib import reload
import ClusterPipeline.models.ClusterProcessing as cp
from copy import deepcopy
import numpy as np
from tensorflow.keras.backend import clear_session

In [ ]:
reload(test_utils)

In [ ]:
group_ids = [40]
groups = await test_utils.extract_cluster_groups(group_ids)

In [ ]:
clusters = await test_utils.get_all_clusters(groups)

In [ ]:
cluster_of_interest_id = 444
cluster_of_interest = [cluster for cluster in clusters if cluster.id == cluster_of_interest_id][0]

In [ ]:
features = groups[0].group_params.training_features

features

In [ ]:
cur_group = groups[0]

In [ ]:
# cur_group = await test_utils.recreate_group(cur_group)

In [ ]:

X_train, y_train, X_test, y_test = cur_group.get_3d_array() 
X_train = X_train.copy()[::-1]
y_train = y_train.copy()[::-1]
X_test = X_test.copy()[::-1]
y_test = y_test.copy()[::-1]
cluster_features = cur_group.group_params.cluster_features
X_feature_dict = cur_group.group_params.X_feature_dict
y_feature_dict = cur_group.group_params.y_feature_dict
training_dict = {}
training_dict['0'] = (0,(X_train, y_train, X_test, y_test))
print(training_dict['0'][1][0].shape)
print(training_dict['0'][1][1].shape)
print(training_dict['0'][1][2].shape)
print(training_dict['0'][1][3].shape)

In [ ]:
print(len(X_feature_dict))
print(len(cur_group.group_params.X_cols))

In [ ]:
print(X_feature_dict)

In [ ]:
next_clusters = test_utils.cluster_a_group(cur_group,training_dict['0'][1],4)
next_label = 2
training_dict['1'] = next_clusters[next_label]

In [ ]:
for i in range(len(next_clusters)): 
    print(next_clusters[i][0].shape)
    plot = test_utils.visualize_cluster_centroid(next_clusters[i][0],cluster_features)
    plot.show()

In [ ]:
next_clusters = test_utils.cluster_a_group(cur_group,training_dict['1'][1],2)
for i in range(len(next_clusters)): 
    print(next_clusters[i][0].shape)
    plot = test_utils.visualize_cluster_centroid(next_clusters[i][0],cluster_features)
    plot.show()
next_label = 0
training_dict['2'] = next_clusters[next_label]

In [ ]:
next_clusters = test_utils.cluster_a_group(cur_group,training_dict['2'][1],3)
for i in range(len(next_clusters)): 
    print(next_clusters[i][0].shape)
    plot = test_utils.visualize_cluster_centroid(next_clusters[i][0],cluster_features)
    plot.show()
next_label = 1
training_dict['3'] = (next_clusters[next_label])

In [48]:
import random
target_features = [] 
# target_features += ['pctChgclose{}_target'.format(i) for i in range(-14, 0) ]
# target_features += ['pctChgclose-0_target']
target_features += ['pctChgclose+{}_target'.format(i) for i in range(1, 16) ]
training_features = random.sample(features, 20)
training_features += ['pctChgclose']


In [49]:
print(training_features)

['pctDiff+bb_high_low40', 'pctDiff+smaVol10_smaVol20', 'pctDiff+bb_low_close20', 'pctDiff+bb_high_close10', 'deriv+smaVol10', 'pctDiff+sma10_sma30', 'pctDiff+ema5_close', 'pctDiff+sma30_sma200', 'pctDiff+sma10_sma50', 'pctChgopen', 'pctDiff+ema10_ema20', 'pctDiff+ema5_ema10', 'pctDiff+ema10_ema100', 'pctDiff+ema10_close', 'pctDiff+bb_high_close60', 'pctDiff+ema30_close', 'pctDiff+sma20_sma50', 'pctDiff+bb_high_low20', 'pctDiff+sma5_sma200', 'opHi', 'pctChgclose']


In [39]:
reload(test_utils)

<module 'test_utils' from '/home/ajp031/ClusterCast/test_utils.py'>

In [50]:
print(target_features)
print(len(target_features))

['pctChgclose+1_target', 'pctChgclose+2_target', 'pctChgclose+3_target', 'pctChgclose+4_target', 'pctChgclose+5_target', 'pctChgclose+6_target', 'pctChgclose+7_target', 'pctChgclose+8_target', 'pctChgclose+9_target', 'pctChgclose+10_target', 'pctChgclose+11_target', 'pctChgclose+12_target', 'pctChgclose+13_target', 'pctChgclose+14_target', 'pctChgclose+15_target']
15


In [51]:
centroid =  training_dict['0'][0]
X_train, y_train, X_test, y_test = training_dict['0'][1]

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_feature_dict)

(6810, 50, 517)
(6810, 55)
(1612, 50, 517)
(1612, 55)
{'pctChgclose+10_target': 0, 'pctChgclose+11_target': 1, 'pctChgclose+12_target': 2, 'pctChgclose+13_target': 3, 'pctChgclose+14_target': 4, 'pctChgclose+15_target': 5, 'pctChgclose+16_target': 6, 'pctChgclose+17_target': 7, 'pctChgclose+18_target': 8, 'pctChgclose+19_target': 9, 'pctChgclose+1_target': 10, 'pctChgclose+20_target': 11, 'pctChgclose+21_target': 12, 'pctChgclose+22_target': 13, 'pctChgclose+23_target': 14, 'pctChgclose+24_target': 15, 'pctChgclose+25_target': 16, 'pctChgclose+2_target': 17, 'pctChgclose+3_target': 18, 'pctChgclose+4_target': 19, 'pctChgclose+5_target': 20, 'pctChgclose+6_target': 21, 'pctChgclose+7_target': 22, 'pctChgclose+8_target': 23, 'pctChgclose+9_target': 24, 'pctChgclose-0_target': 25, 'pctChgclose-10_target': 26, 'pctChgclose-11_target': 27, 'pctChgclose-12_target': 28, 'pctChgclose-13_target': 29, 'pctChgclose-14_target': 30, 'pctChgclose-1_target': 31, 'pctChgclose-2_target': 32, 'pctChgclo

In [52]:
bb_index = X_feature_dict['pctChgclose']
X_train[:,:,bb_index]

array([[-0.13288385, -0.13288385, -0.35688597, ..., -1.4988608 ,
         1.3753952 , -0.28330252],
       [-0.13288385, -0.35688597,  0.5043024 , ...,  1.3753952 ,
        -0.28330252,  0.06215053],
       [-0.35688597,  0.5043024 ,  0.01596742, ..., -0.28330252,
         0.06215053,  0.97756755],
       ...,
       [ 0.14839824,  0.02384049, -0.28681073, ...,  0.01461626,
        -1.5025926 , -0.8410039 ],
       [ 0.02384049, -0.28681073,  0.74642056, ..., -1.5025926 ,
        -0.8410039 , -1.6103275 ],
       [-0.28681073,  0.74642056,  0.27422208, ..., -0.8410039 ,
        -1.6103275 ,  1.4191101 ]], dtype=float32)

In [53]:
# cur_group.run_clustering()
# all_centroids = cur_group.cluster_centers

# closted_centroid_label = test_utils.find_closest_centroid(centroid,all_centroids)
# print(closted_centroid_label)



In [54]:
# print(cur_group.cluster_centers.shape)   


In [55]:
# print(closted_centroid_label)
# test_utils.visualize_cluster_centroid(centroid,cluster_features).show()
# test_utils.visualize_cluster_centroid(all_centroids[closted_centroid_label],cluster_features).show()


In [56]:
print(len(cur_group.clusters))

16


In [57]:

X_train_filtered = test_utils.filter_by_features(X_train, training_features, X_feature_dict)
X_test_filtered = test_utils.filter_by_features(X_test, training_features, X_feature_dict)
y_train_filtered = test_utils.filter_y_by_features(y_train, target_features, y_feature_dict)
y_test_filtered = test_utils.filter_y_by_features(y_test, target_features, y_feature_dict)
print(X_train_filtered.shape)
print(y_train_filtered.shape)
print(X_test_filtered.shape)
print(y_test_filtered.shape)

[10, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 3, 4, 5]
[10, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 3, 4, 5]
(6810, 50, 21)
(6810, 15)
(1612, 50, 21)
(1612, 15)


In [58]:
print(X_train_filtered[0,0,:])

[-0.23253043 -0.21981089  0.9674852   0.63886994 -0.13451391 -0.0611035
 -0.0271259  -0.24894197  0.44689983 -0.46067548 -0.78335243 -1.7458183
  0.14583488 -0.7856218  -0.10269442 -0.49100214  0.91470677 -0.5059422
 -0.39797068  0.13541654 -0.13288385]


In [59]:
training_attention_model, test_attention_model = test_utils.create_attention_model(input_steps=X_train.shape[1],output_steps=len(target_features),features=len(training_features))

2024-03-08 02:15:12.261272: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 02:15:12.263462: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 02:15:12.264960: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [60]:
decoder_weights = test_utils.save_decoder_initial_weights(training_attention_model)

In [61]:
trainined_attention_model = test_utils.train_model(training_attention_model, (X_train, y_train, X_test, y_test), training_features, 
                                                   target_features, X_feature_dict, y_feature_dict,
                                                   epochs = 100, batch_size = 128, lr = 0.001, early_stopping_patience = 15,
                                                   loss = 'mse', shuffle = False)

[10, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 3, 4, 5]
[10, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 3, 4, 5]
Epoch 1/100


2024-03-08 02:15:25.307130: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 02:15:25.309263: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 02:15:25.310776: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

53/54 [============================>.] - ETA: 0s - loss: 0.2527

2024-03-08 02:15:46.577185: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 02:15:46.579555: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 02:15:46.581084: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

54/54 [==============================] - 24s 83ms/step - loss: 0.2539 - val_loss: 0.2693
Epoch 2/100
54/54 [==============================] - 3s 47ms/step - loss: 0.2511 - val_loss: 0.2691
Epoch 3/100
54/54 [==============================] - 3s 48ms/step - loss: 0.2506 - val_loss: 0.2696
Epoch 4/100
54/54 [==============================] - 3s 47ms/step - loss: 0.2500 - val_loss: 0.2684
Epoch 5/100
54/54 [==============================] - 3s 48ms/step - loss: 0.2494 - val_loss: 0.2681
Epoch 6/100
54/54 [==============================] - 3s 47ms/step - loss: 0.2490 - val_loss: 0.2690
Epoch 7/100
54/54 [==============================] - 3s 48ms/step - loss: 0.2493 - val_loss: 0.2684
Epoch 8/100
54/54 [==============================] - 3s 47ms/step - loss: 0.2485 - val_loss: 0.2687
Epoch 9/100
54/54 [==============================] - 3s 49ms/step - loss: 0.2479 - val_loss: 0.2697
Epoch 10/100
54/54 [==============================] - 3s 47ms/step - loss: 0.2481 - val_loss: 0.2700
Epoch 11/1

In [62]:
num_days = 15
attention_model_accuracy, attention_results, predicted_y, attention_weights = test_utils.eval_model(X_test_filtered, y_test_filtered, trainined_attention_model,num_days, test_model = False)

2024-03-08 02:16:40.075572: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 02:16:40.078118: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 02:16:40.079842: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

51/51 [==============================] - 2s 11ms/step
<class 'numpy.ndarray'> 1612
(1612, 15)
15


In [63]:
for layer in trainined_attention_model.layers:
    if 'freeze' in layer.name:
        layer.trainable = False
        continue
    if 'input' in layer.name:
        continue
    layer.set_weights(decoder_weights[layer.name])

In [64]:
reload(test_utils)

<module 'test_utils' from '/home/ajp031/ClusterCast/test_utils.py'>

In [65]:
print(attention_model_accuracy)
test_utils.visualize_future_distribution(attention_results)

Cluster Number:
Accuracy1D 52.92% PredictedRet: 0.016839036718010902 ActRet: 0.025182369734199888
Accuracy2D 51.3% PredictedRet: 0.0321800522506237 ActRet: 0.05008982746485186
Accuracy3D 52.48% PredictedRet: 0.048899341374635696 ActRet: 0.07498872419414815
Accuracy4D 54.22% PredictedRet: 0.06804940849542618 ActRet: 0.10000477679541495
Accuracy5D 55.21% PredictedRet: 0.0891897901892662 ActRet: 0.1255876312609941
Accuracy6D 56.02% PredictedRet: 0.11115375906229019 ActRet: 0.15062731337494092
Accuracy7D 56.2% PredictedRet: 0.133416086435318 ActRet: 0.1762633253339269
Accuracy8D 57.44% PredictedRet: 0.1558619737625122 ActRet: 0.20196269108792803
Accuracy9D 55.58% PredictedRet: 0.1773906797170639 ActRet: 0.2273711794054265
Accuracy10D 55.77% PredictedRet: 0.1995932012796402 ActRet: 0.25281603253703905
Accuracy11D 56.39% PredictedRet: 0.22058965265750885 ActRet: 0.2776886298930418
Accuracy12D 56.33% PredictedRet: 0.24116171896457672 ActRet: 0.3020449397992603
Accuracy13D 56.89% PredictedRet:

In [66]:
centroid =  training_dict['2'][0]
X_train_smaller, y_train_smaller, X_test_smaller, y_test_smaller= training_dict['2'][1]
# X_train_larger = np.concatenate((X_train, X_train_larger), axis=0)
# y_train_larger = np.concatenate((y_train, y_train_larger), axis=0)
# X_test_larger = np.concatenate((X_test, X_test_larger), axis=0)
# y_test_larger = np.concatenate((y_test, y_test_larger), axis=0)

# X_train_larger = np.concatenate((X_train, X_train_larger), axis=0)
# y_train_larger = np.concatenate((y_train, y_train_larger), axis=0)
# X_test_larger = np.concatenate((X_test, X_test_larger), axis=0)
# y_test_larger = np.concatenate((y_test, y_test_larger), axis=0)

X_train_smaller_filtered = test_utils.filter_by_features(X_train_smaller, training_features, X_feature_dict)
X_test_smaller_filtered = test_utils.filter_by_features(X_test_smaller, training_features, X_feature_dict)
y_train_smaller_filtered = test_utils.filter_y_by_features(y_train_smaller, target_features, y_feature_dict)
y_test_smaller_filtered = test_utils.filter_y_by_features(y_test_smaller, target_features, y_feature_dict)



print(X_train_smaller_filtered.shape)
print(y_train_smaller_filtered.shape)
print(X_test_smaller_filtered.shape)
print(y_test_smaller_filtered.shape)

[10, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 3, 4, 5]
[10, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 3, 4, 5]
(1145, 50, 21)
(1145, 15)
(217, 50, 21)
(217, 15)


In [67]:
larger_train_model = test_utils.train_model(trainined_attention_model, (X_train_smaller, y_train_smaller, X_test_smaller, y_test_smaller), training_features, 
                                                   target_features, X_feature_dict, y_feature_dict,
                                                   epochs = 150, batch_size = 8, lr = 0.0001, early_stopping_patience = 30,
                                                   loss = 'mse')

[10, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 3, 4, 5]
[10, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 3, 4, 5]
Epoch 1/150


2024-03-08 02:17:00.733491: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 02:17:00.735511: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 02:17:00.737071: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

143/144 [============================>.] - ETA: 0s - loss: 0.2237

2024-03-08 02:17:23.600341: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 02:17:23.602497: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 02:17:23.604030: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

144/144 [==============================] - 26s 45ms/step - loss: 0.2242 - val_loss: 0.2422
Epoch 2/150
144/144 [==============================] - 5s 32ms/step - loss: 0.2213 - val_loss: 0.2434
Epoch 3/150
144/144 [==============================] - 5s 32ms/step - loss: 0.2200 - val_loss: 0.2441
Epoch 4/150
144/144 [==============================] - 5s 32ms/step - loss: 0.2191 - val_loss: 0.2447
Epoch 5/150
144/144 [==============================] - 5s 32ms/step - loss: 0.2182 - val_loss: 0.2453
Epoch 6/150
144/144 [==============================] - 5s 32ms/step - loss: 0.2175 - val_loss: 0.2460
Epoch 7/150
144/144 [==============================] - 5s 32ms/step - loss: 0.2167 - val_loss: 0.2465
Epoch 8/150
144/144 [==============================] - 5s 33ms/step - loss: 0.2161 - val_loss: 0.2471
Epoch 9/150
144/144 [==============================] - 5s 32ms/step - loss: 0.2154 - val_loss: 0.2475
Epoch 10/150
144/144 [==============================] - 5s 32ms/step - loss: 0.2148 - val_los

In [68]:
large_data_set_accuracy, large_data_set_results, predicted_y, attention_weights = test_utils.eval_model(X_test_smaller_filtered, y_test_smaller_filtered, test_attention_model, num_days)

2024-03-08 02:19:48.683256: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 02:19:48.685273: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 02:19:48.686689: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

7/7 [==============================] - 2s 12ms/step
<class 'list'> 2
(217, 15)
15


In [ ]:
benchmark_model = test_utils.create_attention_model(input_steps=X_train.shape[1],output_steps=len(target_features),features=len(training_features))
benchark_model = test_utils.train_model(benchmark_model, (X_train_smaller, y_train_smaller, X_test_smaller, y_test_smaller), training_features,
                                        target_features, X_feature_dict, y_feature_dict,
                                        epochs = 250, batch_size = 32, lr = 0.001, early_stopping_patience = 30,
                                        loss = 'mse')

In [ ]:
# benchark_model = test_utils.train_model(benchmark_model, (X_train_smaller, y_train_smaller, X_test_smaller, y_test_smaller), training_features,
#                                         target_features, X_feature_dict, y_feature_dict,
#                                         epochs = 250, batch_size = 16, lr = 0.0001, early_stopping_patience = 30,
#                                         loss = test_utils.custom_profit_loss_percent_change)

In [ ]:
bench_model_accuracy, bench_results = test_utils.eval_model(X_test_smaller_filtered, y_test_smaller_filtered, benchmark_model)

In [69]:
print("FINE TUNED MODEL")
print(large_data_set_accuracy)
test_utils.visualize_future_distribution(large_data_set_results)

FINE TUNED MODEL
Cluster Number:
Accuracy1D 53.92% PredictedRet: 0.012565530836582184 ActRet: 0.022200928232739874
Accuracy2D 54.84% PredictedRet: 0.017477378249168396 ActRet: 0.05188380900667423
Accuracy3D 53.92% PredictedRet: 0.011842354200780392 ActRet: 0.08015299505180752
Accuracy4D 55.76% PredictedRet: 0.004529100377112627 ActRet: 0.10944378520641047
Accuracy5D 53.92% PredictedRet: -0.011208697222173214 ActRet: 0.11146140041061535
Accuracy6D 52.53% PredictedRet: -0.026374008506536484 ActRet: 0.1287064976895803
Accuracy7D 50.69% PredictedRet: -0.04698134586215019 ActRet: 0.14676528867583052
Accuracy8D 53.0% PredictedRet: -0.06442689150571823 ActRet: 0.16085434023908893
Accuracy9D 53.0% PredictedRet: -0.07888621091842651 ActRet: 0.1885045513377658
Accuracy10D 52.53% PredictedRet: -0.08644343167543411 ActRet: 0.21240373990983458
Accuracy11D 52.07% PredictedRet: -0.0828452780842781 ActRet: 0.24712123615615722
Accuracy12D 52.53% PredictedRet: -0.07218824326992035 ActRet: 0.297905868009

: 

In [ ]:
print("BENCHMARK MODEL")
print(bench_model_accuracy)
test_utils.visualize_future_distribution(bench_results)

In [ ]:
clear_session()